---
title: "`primate` quickstart"
---




`primate` is a package that contains a variety of algorithms for estimating quantities from matrix functions, with a focus 
on implicit matrix representations and support for common quantities of interest, such as the trace or the diagonal.

Below is a quick introduction to: 

- Trace estimation
- Diagonal estimation 
- Matrix function approximation
- Configuring the output 
- Configuring the execution

<br/>


In [ ]:
#| echo: false
#| output: false
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
import numpy as np
np.random.seed(1234)

## Trace estimation 

A variety of _trace_ estimators are available in the `primate.trace` module. These algorithms estimate the quantity: 

$$ \mathrm{tr}(A) = \sum\limits_{i=1}^n A_{ii} = \sum\limits_{i=1}^n \lambda_i $$

The following example demonstrates a variety of trace estimators, including the Girard-Hutchinson estimator (`hutch`), the 
improved Hutch++ (`hutchpp`), and XTrace (`xtrace`):


In [ ]:
from primate.trace import hutch, hutchpp, xtrace
from primate.random import symmetric
rng = np.random.default_rng(1234)      # for reproducibility 
A = symmetric(150, pd=True, seed=rng)  # random PD matrix 

print(f"Trace   :  {A.trace():6f}")   ## Actual trace
print(f"Hutch   :  {hutch(A):6f}")    ## Crude Monte-Carlo estimator
print(f"Hutch++ :  {hutchpp(A):6f}")  ## Monte-Carlo estimator w/ deflation 
print(f"XTrace  :  {xtrace(A):6f}")   ## Epperly's algorithm

Here, `A` can be a numpy matrix, a sparse matrix, or a `LinearOperator`. If the spectral sum of interest is not just the sum of the eigenvalues, but rather the sum under composition with some spectral function $f(\lambda)$, i.e. the quantity of interest is of the form: 

$$ \mathrm{tr}(f(A)) = \sum\limits_{i=1}^n f(A)_{ii} = \sum\limits_{i=1}^n f(\lambda_i) $$

Then you may alternatively use the `matrix_function` API to construct a `LinearOperator` by passing in the corresponding spectral function as a `Callable` or by passing a string representing the name of one of the following built-in spectral functions:

| **name**       | **matrix function**       | **Applications**                          | **numpy call**                     |
|----------------|---------------------------|-------------------------------------------|------------------------------------|
| identity       | $A$                      | Basic matrix operations                   | `A`                                |
| log            | $\log(A)$                | Determinant, entropy-like measures        | `logm(A)`                         |
| exp            | $e^A$                    | Dynamical systems, graph diffusion        | `expm(A)`                         |
| inv            | $A^{-1}$                 | Stability analysis, linear systems        | `inv(A)`                          |
| sign           | $\text{sgn}_\epsilon(A)$ | Rank approximation, low-rank modeling     | `U @ U.T`                         |
| sqrt           | $A^{1/2}$                | Diffusion, kernel methods                 | `sqrtm(A)`                        |
| square         | $A^2$                    | Energy measures, stability                | `A @ A`                           |
| PCA            | $P_A$ (eigenspace)       | Dim. reduction, feature extraction        | Custom projection matrix          |
| Tikhonov       | $(A + \lambda I)^{-1}$   | Regularized inversion, stability          | `inv(A + lambda * eye(n))`        |
| heat           | $e^{-tA}$                | Diffusion on graphs, spectral clustering  | `expm(-t * A)`                    |
| pagerank       | $(I - \alpha A)^{-1}v$   | Network centrality, web ranking           | Iterative solver                  |

<br/>

For example, one might compute the log-determinant of a positive definite matrix as follows:


In [ ]:
from primate.operators import matrix_function, IdentityOperator
M = matrix_function(A, fun="log") # or fun=np.log 

print(f"logdet(A) :  {np.log(np.linalg.det(A)):6f}")
print(f"tr(log(A)):  {np.sum(np.log(np.linalg.eigvalsh(A))):6f}")
print(f"Hutch     :  {hutch(M):6f}")
print(f"Hutch++   :  {hutchpp(M):6f}")
print(f"XTrace    :  {xtrace(M):6f}")

## Diagonal estimation 

The diagonals of matrices and matrix functions (implicitly or explicitly represented) can also be estimated via nearly identical API used for the trace.  


In [ ]:
from primate.estimators import arr_summary
from primate.diagonal import diag, xdiag

d1 = A.diagonal()
d2 = diag(A, rtol=1e-4)
d3 = xdiag(A)

print(f"Diagonal (true): {arr_summary(d1)}")
print(f"Diagonal Hutch : {arr_summary(d2)}")
print(f"Diagonal XDiag : {arr_summary(d3)}")

## Matrix function approximation 

In `primate`, the matrix function $f(A)$ is not constructed explicitly but instead the action $v \mapsto f(A)v$ is approximated with a fixed-degree Krylov expansion. This can be useful when, for example, the matrix $A$ itself is so large that the corresponding (typically dense) matrix function $f(A) \in \mathbb{R}^{n \times n}$ simply is too large to be explicitly represented. If you just want to approximate the action of a matrix function for a single vector $v \in \mathbb{R}^n$, simply supply the vector and the matrix alongside the `matrix_function` call: 


In [ ]:
from primate.operators import matrix_function
v = np.random.uniform(size=A.shape[0])
y = matrix_function(A, fun=np.exp, v=v)
print(f"f(A)v = {arr_summary(y.ravel())}")

Alternatively, if you prefer an object-oriented approach (or you plan on doing multiple matvecs), you can construct a `MatrixFunction` instance and use it like any other `LinearOperator`: 


In [ ]:
from primate.operators import MatrixFunction
ExpA = MatrixFunction(A, fun=np.exp)
y = ExpA @ v
print(f"exp(A)v = {arr_summary(y)}")

If you don't supply a vector `v` to the `matrix_function` call, a `MatrixFunction` instance is constructed using whatever additional arguments are passed in and returned. Note some function specializations are inherently more difficult to approximate and can depend on the smoothness of $f$ and the conditioning of the corresponding operator $f(A)$; in general, a `MatrixFunction` instance with degree $k$ approximates the action $v \mapsto f(A)v$ about as well as the operator $p(A)$, where $p$ is a degree $2k-1$ polynomial interpolant of $f$. 


In [ ]:
from scipy.linalg import expm
ExpA = expm(A)
ExpA0 = MatrixFunction(A, fun=np.exp, deg=5, orth=0)
ExpA1 = MatrixFunction(A, fun=np.exp, deg=20, orth=0)
ExpA2 = MatrixFunction(A, fun=np.exp, deg=50, orth=50)

w = ExpA @ v
x = ExpA0 @ v
y = ExpA1 @ v 
z = ExpA2 @ v

print(f"Deg-5 approx error  (no reorth.)   : {np.linalg.norm(w - x)}")
print(f"Deg-20 approx error (no reorth.)   : {np.linalg.norm(w - y)}")
print(f"Deg-50 approx error (full reorth.) : {np.linalg.norm(w - z)}")

As you can see, for smoother matrix functions (like $\mathrm{exp}(A)$), even a low degree Krylov expansion can be more than sufficient for many application purposes---all without any re-orthogonalization! See the matrix function guide for more background on this. 

## Configuring the output

<!-- By default, the various estimators offered by `primate` simply return the estimated quantity under reasonable default parameter settings. However, in many applications one would like to have greater control over both the computation itself and the type of information collected during execution.  -->

Passing `full=True` returns additional information about the computation in the form of `EstimatorResult` (along with the estimate itself), which contains information about execution itself, convergence information of the estimator, and other status messages. 

For example, with the default `converge="confidene"` criterion, the margin of error of a default-constructed confidence interval is returned: 


In [ ]:
est, info = hutch(A, converge="confidence", full=True)
print(info.message)

A more visual way of viewing the sample values and the corresponding estimate as a function of the sample size is to plot the sequence with 
the `figure_sequence` function (note this requires saving the samples with `record=True`):


In [ ]:
#| echo: true
from primate.plotting import figure_sequence

est, info = hutch(A, full=True, record=True)
p = figure_sequence(info.estimator.values)
show(p)

You can also pass a callback function, which receives as its only argument an `EstimatorResult` instance. 
This can be useful for quickly monitoring convergence information. 


In [ ]:
def hutch_callback(result: EstimatorResult):
	if (result.nit % 150) == 0:
		print(result.criterion.message(result.estimator))

est, info = hutch(A, converge="confidence", callback=hutch_callback)

<!-- For example, consider applying the `sign` function at some tolerance level $\epsilon$:

$$ \mathrm{sign}_\epsilon(x) = \begin{align} 1 & x \geq \epsilon \\ 0 & \text{otherwise} \end{align} $$

The spectral sum defined by applying this function corresponds to the numerical rank of the operator, which is an interesting and sometimes useful quantity to estimate. However, the sign function itself is discontinuous (at $\epsilon$) and otherwise not smooth; thus, it cannot be well approximated by any polynomial of finite degree. Non-analytic functions such as these can prove difficult to approximate, as shown below:


ew = np.sort(np.random.uniform(size=150, low=0, high=1))
ew[:30] = 0.0
A = symmetric(150, ew = ew, pd = False)
M = matrix_function(A, fun="numrank")

print(f"numrank(A): {np.linalg.matrix_rank(A)}")
print(f"GR approx:  {hutch(M)}")
print(f"Hutch++ :   {hutchpp(M)}")
print(f"XTrace:     {xtrace(M)}")
``` -->
<!-- from timeit import timeit
timeforlan = timeit(lambda: ExpA @ v, number=100)/100

timeforexpmA = timeit(lambda: expm(A), number=100)/100
B = expm(A)
timeforexpmAv = timeit(lambda: B @ v, number=100)/100

sample_index = 1 + np.arange(30)
timeforsp = timeforexpmA + timeforexpmAv * sample_index
timeformf = timeforlan * sample_index

p = figure(width=350, height=200)

p.line(sample_index, timeforsp, color='red')
p.scatter(sample_index, timeforsp, color='red',  size=1)
p.line(sample_index, timeformf, color='blue')
p.scatter(sample_index, timeformf, color='blue', size=1)
show(p) -->